In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fancyimpute import KNN
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
import seaborn as sns
from sklearn.model_selection import KFold
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

## 读取数据

查看数据字段及类型

In [ ]:
train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')

print(train_df.shape, test_df.shape)
print(train_df.dtypes, test_df.dtypes)

In [ ]:
train_df

## 数据分析

缺失值处理

In [ ]:
def do_nan(train_df,test_df,method = None):
    #口服耐糖量测试为-1，也应该为null
    train_df['口服耐糖量测试'] = train_df['口服耐糖量测试'].replace(-1, np.nan)
    test_df['口服耐糖量测试'] = test_df['口服耐糖量测试'].replace(-1, np.nan)

    train_df['口服耐糖量测试'] = train_df['口服耐糖量测试'].replace(0, np.nan)
    test_df['口服耐糖量测试'] = test_df['口服耐糖量测试'].replace(0, np.nan)
    if method is not None:
        train_df['胰岛素释放实验'] = train_df['胰岛素释放实验'].replace(0, np.nan)
        test_df['胰岛素释放实验'] = test_df['胰岛素释放实验'].replace(0, np.nan)

        train_df['肱三头肌皮褶厚度'] = train_df['肱三头肌皮褶厚度'].replace(0, np.nan)
        test_df['肱三头肌皮褶厚度'] = test_df['肱三头肌皮褶厚度'].replace(0, np.nan)

    train_df['体重指数'] = train_df['体重指数'].replace(0, np.nan)
    test_df['体重指数'] = test_df['体重指数'].replace(0, np.nan)
    
        # 训练集缺失值情况
    print(train_df.isnull().mean(0))
    # 训练集缺失值情况
    print(test_df.isnull().mean(0))

    
    
    #填充缺失值处理
    #下列三个字段比例较低，可以直接填充
    train_df['舒张压'].fillna(89, inplace=True)
    test_df['舒张压'].fillna(89, inplace=True)

    train_df['口服耐糖量测试'].fillna(6,inplace = True)
    test_df['口服耐糖量测试'].fillna(6,inplace = True)

    train_df['体重指数'].fillna(38,inplace = True)
    test_df['体重指数'].fillna(38,inplace = True)


    #胰岛素释放实验	肱三头肌皮褶厚度    字段缺失比例较高，试试统计填充（多元线性回归）
    all_df = pd.concat([train_df,test_df])
    all_df.drop(['患有糖尿病标识'],axis=1,inplace=True)
    all_df.dropna(inplace=True)
    if method == 'lg':
        lg_g3 = LinearRegression()
        lg_yd = LinearRegression()
        lg_features = [f for f in all_df.columns if f not in ['编号','肱三头肌皮褶厚度','胰岛素释放实验']]

        lg_g3.fit(all_df[lg_features],all_df['肱三头肌皮褶厚度'])
        lg_yd.fit(all_df[lg_features],all_df['胰岛素释放实验'])

        for index,bool in enumerate (train_df['胰岛素释放实验'].isnull()):
            if  bool:
                train_df.loc[index,'胰岛素释放实验']  = lg_yd.predict(np.array(train_df.loc[index,lg_features]).reshape(1, -1))
                
            
        for index,bool in enumerate (train_df['肱三头肌皮褶厚度'].isnull()):
            if  bool:
                train_df.loc[index,'肱三头肌皮褶厚度']  = lg_g3.predict(np.array(train_df.loc[index,lg_features]).reshape(1, -1))
                
        for index,bool in enumerate (test_df['胰岛素释放实验'].isnull()):
            if  bool:
                test_df.loc[index,'胰岛素释放实验']  = lg_yd.predict(np.array(test_df.loc[index,lg_features]).reshape(1, -1))
                
            
        for index,bool in enumerate (test_df['肱三头肌皮褶厚度'].isnull()):
            if  bool:
                test_df.loc[index,'肱三头肌皮褶厚度']  = lg_g3.predict(np.array(test_df.loc[index,lg_features]).reshape(1, -1))        
    
    #试试knn拟合
    if method == 'knn':
        train_df = pd.DataFrame(KNN(k=5).fit_transform(train_df),columns= train_df.columns)
        test_df = pd.DataFrame(KNN(k=5).fit_transform(test_df),columns= test_df.columns)
    
    

        

    return train_df,test_df        

字段类型分析

In [ ]:
train_df.describe()

In [ ]:
#计算相关性
train_df.corr()

In [ ]:
for column in train_df.columns:
    if column == '编号':
        continue
    if column == '患有糖尿病标识':
        continue
    plt.figure()
    for i in [0,1]:
        train_df[column][train_df['患有糖尿病标识']==i].hist()
    plt.title("{}".format(column))    
    plt.savefig("fig/{}.png".format(column))

## 逻辑回归尝试

In [ ]:
train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')

train_df,test_df = do_nan(train_df,test_df)

#划分训练集、验证集
features = [f for f in train_df.columns if f not in ['编号','出生年份','患有糖尿病标识']]
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)


In [ ]:
# 训练集表现
l1_train_predict = []
l2_train_predict = []

# 测试集表现
l1_test_predict = []
l2_test_predict = []

for c in np.linspace(0.01, 5, 50) :
    lr_l1 = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l1", C=c, solver="liblinear", max_iter=2000))
    lr_l2 = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l2", C=c, solver="liblinear", max_iter=2000))
    
    # 训练模型，记录L1正则化模型在训练集测试集上的表现
    lr_l1.fit(x_train, y_train)
    l1_train_predict.append(accuracy_score(lr_l1.predict(x_train), y_train))
    l1_test_predict.append(accuracy_score(lr_l1.predict(x_test), y_test))
    
    # 记录L2正则化模型的表现
    lr_l2.fit(x_train, y_train)
    l2_train_predict.append(accuracy_score(lr_l2.predict(x_train), y_train))
    l2_test_predict.append(accuracy_score(lr_l2.predict(x_test), y_test))
    
data = [l1_train_predict, l2_train_predict, l1_test_predict, l2_test_predict]
label = ['l1_train', 'l2_train', 'l1_test', "l2_test"]
color = ['red', 'green', 'orange', 'blue']

plt.figure(figsize=(12, 6))
for i in range(4) :
    plt.plot(np.linspace(0.01, 5, 50), data[i], label=label[i], color=color[i])

plt.legend(loc="best")
plt.show()


In [ ]:
for c in np.linspace(0.01, 2, 50):
    best_model = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l2", C=0.614, solver="liblinear", max_iter=2000))
    best_model.fit(x_train,y_train)
    acc = accuracy_score(best_model.predict(x_test),y_test)
    print(c,acc)

基于最优模型进行预测

In [ ]:
#取出最优参数，进行训练、预测
best_model = make_pipeline(
    MinMaxScaler(),
    LogisticRegression(penalty="l2", C=0.614, solver="liblinear", max_iter=2000)
)

best_model.fit(train_df[features],train_df['患有糖尿病标识'])

test_df['label'] = best_model.predict(test_df[features])
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit.csv', index=None)

## 特征工程

统计每个性别对应的体重指数、舒张压的平均值，并构建新特征

In [ ]:
def do_feature(train_df,test_df,sex, family):
    if sex :
        df_sex = train_df[['体重指数','舒张压']].groupby(train_df['性别']).mean()

        train_df['体重指数_diff'] = [train_df.loc[i,'体重指数'] - df_sex.loc[train_df.loc[i,'性别'],'体重指数'] for i in range(len(train_df))]
        train_df['舒张压_diff'] = [train_df.loc[i,'舒张压'] - df_sex.loc[train_df.loc[i,'性别'],'舒张压'] for i in range(len(train_df))]
        
        test_df['体重指数_diff'] = [test_df.loc[i,'体重指数'] - df_sex.loc[test_df.loc[i,'性别'],'体重指数'] for i in range(len(test_df))]
        test_df['舒张压_diff'] = [test_df.loc[i,'舒张压'] - df_sex.loc[test_df.loc[i,'性别'],'舒张压'] for i in range(len(test_df))]
    
    if family :
        df_family = train_df[['口服耐糖量测试','胰岛素释放实验']].groupby(train_df['糖尿病家族史']).mean()

        for column in ['口服耐糖量测试','胰岛素释放实验']:
            train_df[column +'_diff'] = [train_df.loc[i,column] - df_family.loc[train_df.loc[i,'糖尿病家族史'],column] for i in range(len(train_df))]
            test_df[column +'_diff'] = [test_df.loc[i,column] - df_family.loc[test_df.loc[i,'糖尿病家族史'],column] for i in range(len(test_df))]
            
            
    train_df['体重指数_round'] = train_df['体重指数'] // 10
    test_df['体重指数_round'] = train_df['体重指数'] // 10


    train_df['糖尿病家族史'] = train_df['糖尿病家族史'].astype('category')
    test_df['糖尿病家族史'] = train_df['糖尿病家族史'].astype('category')

    train_df['性别'] = train_df['性别'].astype('category')
    test_df['性别'] = train_df['性别'].astype('category')

    #将糖尿病家族史转换为类别变量
    dict_糖尿病家族史 = {
        '无记录': 0,
        '叔叔或姑姑有一方患有糖尿病': 1,
        '叔叔或者姑姑有一方患有糖尿病': 1,
        '父母有一方患有糖尿病': 2
    }
    train_df['糖尿病家族史'] = train_df['糖尿病家族史'].map(dict_糖尿病家族史)
    test_df['糖尿病家族史'] = test_df['糖尿病家族史'].map(dict_糖尿病家族史)

    #出生年份转换
    train_df['年龄'] = 2022 - train_df['出生年份']
    test_df['年龄'] = 2022 - test_df['出生年份']

    
    train_df['口服耐糖量测试_diff'] = train_df['口服耐糖量测试'] - train_df.groupby('糖尿病家族史').transform('mean')['口服耐糖量测试']
    test_df['口服耐糖量测试_diff'] = test_df['口服耐糖量测试'] - test_df.groupby('糖尿病家族史').transform('mean')['口服耐糖量测试']
    return train_df,test_df
    

In [ ]:
train_df.groupby('糖尿病家族史').transform('mean')

In [ ]:
train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')
train_df,test_df = do_nan(train_df,test_df)
train_df,test_df = do_feature(train_df,test_df,True,False)

#划分训练集、验证集
features = [f for f in train_df.columns if f not in ['编号','出生年份','患有糖尿病标识']]
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)

In [ ]:
# 训练集表现
l1_train_predict = []
l2_train_predict = []

# 测试集表现
l1_test_predict = []
l2_test_predict = []

for c in np.linspace(0.01, 5, 50) :
    lr_l1 = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l1", C=c, solver="liblinear", max_iter=2000))
    lr_l2 = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l2", C=c, solver="liblinear", max_iter=2000))
    
    # 训练模型，记录L1正则化模型在训练集测试集上的表现
    lr_l1.fit(x_train, y_train)
    l1_train_predict.append(accuracy_score(lr_l1.predict(x_train), y_train))
    l1_test_predict.append(accuracy_score(lr_l1.predict(x_test), y_test))
    
    # 记录L2正则化模型的表现
    lr_l2.fit(x_train, y_train)
    l2_train_predict.append(accuracy_score(lr_l2.predict(x_train), y_train))
    l2_test_predict.append(accuracy_score(lr_l2.predict(x_test), y_test))
    
data = [l1_train_predict, l2_train_predict, l1_test_predict, l2_test_predict]
label = ['l1_train', 'l2_train', 'l1_test', "l2_test"]
color = ['red', 'green', 'orange', 'blue']

plt.figure(figsize=(12, 6))
for i in range(4) :
    plt.plot(np.linspace(0.01, 5, 50), data[i], label=label[i], color=color[i])

plt.legend(loc="best")
plt.show()

In [ ]:
for c in np.linspace(0.01, 2, 100):
    best_model = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l2", C=c, solver="liblinear", max_iter=2000))
    best_model.fit(x_train,y_train)
    acc = accuracy_score(best_model.predict(x_test),y_test)
    print(c,acc)

## 特征筛选

In [ ]:
train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')
train_df,test_df = do_nan(train_df,test_df)
train_df,test_df = do_feature(train_df,test_df,True,False)

#划分训练集、验证集
features = [f for f in train_df.columns if f not in ['编号','出生年份','患有糖尿病标识']]
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)

In [ ]:
# 构建模型
model = make_pipeline(
    MinMaxScaler(),
    DecisionTreeClassifier()
)
#拟合结果
model.fit(x_train,y_train)
#查看特征重要度
for index, importance in enumerate(model.named_steps['decisiontreeclassifier'].feature_importances_):
    print(index, importance)
model.named_steps['decisiontreeclassifier'].feature_importances_

In [ ]:
features = ['体重指数','舒张压','口服耐糖量测试', '胰岛素释放实验','肱三头肌皮褶厚度',]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)

In [ ]:
# 训练集表现
l1_train_predict = []
l2_train_predict = []

# 测试集表现
l1_test_predict = []
l2_test_predict = []

for c in np.linspace(0.01, 5, 50) :
    lr_l1 = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l1", C=c, solver="liblinear", max_iter=2000))
    lr_l2 = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l2", C=c, solver="liblinear", max_iter=2000))
    
    # 训练模型，记录L1正则化模型在训练集测试集上的表现
    lr_l1.fit(x_train, y_train)
    l1_train_predict.append(accuracy_score(lr_l1.predict(x_train), y_train))
    l1_test_predict.append(accuracy_score(lr_l1.predict(x_test), y_test))
    
    # 记录L2正则化模型的表现
    lr_l2.fit(x_train, y_train)
    l2_train_predict.append(accuracy_score(lr_l2.predict(x_train), y_train))
    l2_test_predict.append(accuracy_score(lr_l2.predict(x_test), y_test))
    
data = [l1_train_predict, l2_train_predict, l1_test_predict, l2_test_predict]
label = ['l1_train', 'l2_train', 'l1_test', "l2_test"]
color = ['red', 'green', 'orange', 'blue']

plt.figure(figsize=(12, 6))
for i in range(4) :
    plt.plot(np.linspace(0.01, 5, 50), data[i], label=label[i], color=color[i])

plt.legend(loc="best")
plt.show()

In [ ]:
for c in np.linspace(0.01, 2, 100):
    best_model = make_pipeline(MinMaxScaler(),
    LogisticRegression(penalty="l2", C=c, solver="liblinear", max_iter=2000))
    best_model.fit(x_train,y_train)
    acc = accuracy_score(best_model.predict(x_test),y_test)
    print(c,acc)

## 高阶树模型

In [ ]:
train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')
train_df,test_df = do_nan(train_df,test_df)
train_df,test_df = do_feature(train_df,test_df,False,False)

#划分训练集、验证集
features = [f for f in train_df.columns if f not in ['编号','出生年份','患有糖尿病标识']]
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)

In [ ]:
#构建模型
clf = lgb.LGBMClassifier(
    max_depth=3, 
    n_estimators=4000, 
    n_jobs=-1, 
    verbose=-1,
    verbosity=-1,
    learning_rate=0.1,
)

#模型训练
clf.fit(x_train,y_train)

print(accuracy_score(y_test,clf.predict(x_test)))

test_df['label'] = clf.predict(test_df[features])
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit.csv', index=None)

调参

In [ ]:
#第一步：确定残差树的数目

#给定一些初始的参数
params = {    
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metrics': 'binary_logloss',
          'nthread':4,
          'learning_rate':0.1,
          'num_leaves':30, 
          'max_depth': 5,   
          'subsample': 0.8, 
          'colsample_bytree': 0.8, 
    }
#训练集
data_train = lgb.Dataset(x_train, y_train)

cv_results = lgb.cv(params, data_train, num_boost_round=1000, nfold=5, stratified=False, shuffle=True, metrics='binary_logloss',early_stopping_rounds=50,seed=0)
print('best n_estimators:', len(cv_results['binary_logloss-mean']))
print('best cv score:', pd.Series(cv_results['binary_logloss-mean']).min())


In [ ]:
#第二步：确定max_depth 和num_leaves

params_test1={'max_depth': range(3,8,1), 'num_leaves':range(5, 100, 5)}

gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.1, n_estimators=94, bagging_fraction = 0.8,feature_fraction = 0.8), 
                       param_grid = params_test1, scoring='accuracy',cv=5,n_jobs=-1)
gsearch1.fit(x_train,y_train)
gsearch1.best_params_, gsearch1.best_score_


In [ ]:
#第三步：确定min_data_in_leaf 、 max_bin

params_test2={'max_bin': range(5,256,10), 'min_data_in_leaf':range(1,102,10)}
gsearch2 = GridSearchCV(estimator = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.1, n_estimators=94, max_depth=3, num_leaves=10,bagging_fraction = 0.8,feature_fraction = 0.8), 
                       param_grid = params_test2, scoring='accuracy',cv=5,n_jobs=-1)
gsearch2.fit(x_train,y_train)
gsearch2.best_params_, gsearch2.best_score_


In [ ]:
#第四步 确定feature_fraction、bagging_fraction、bagging_freq

params_test3={'feature_fraction': [0.6,0.7,0.8,0.9,1.0],
              'bagging_fraction': [0.6,0.7,0.8,0.9,1.0],
              'bagging_freq': range(0,81,10)
}
              
gsearch3 = GridSearchCV(estimator = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.1, n_estimators=94, max_depth=3, num_leaves=10,max_bin=175,min_data_in_leaf=101), 
                       param_grid = params_test3, scoring='accuracy',cv=5,n_jobs=-1)
gsearch3.fit(x_train,y_train)
gsearch3.best_params_, gsearch3.best_score_


In [ ]:
#第五步 确定lambda_l1和lambda_l2
params_test4={'lambda_l1': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0],
              'lambda_l2': [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]
}
              
gsearch4 = GridSearchCV(estimator = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.1, n_estimators=94, max_depth=3, num_leaves=10,max_bin=175,min_data_in_leaf=101,bagging_fraction=0.6,bagging_freq= 0, feature_fraction= 0.8), 
                       param_grid = params_test4, scoring='accuracy',cv=5,n_jobs=-1)
gsearch4.fit(x_train,y_train)
gsearch4.best_params_, gsearch4.best_score_


In [ ]:
#第六步：确定 min_split_gain
params_test5={'min_split_gain':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}

gsearch5 = GridSearchCV(estimator = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.1, n_estimators=94, max_depth=3, num_leaves=10,max_bin=175,min_data_in_leaf=101,bagging_fraction=0.6,bagging_freq= 0, feature_fraction= 0.8,
                                                       ambda_l1=0.7,lambda_l2=0.7), 
                       param_grid = params_test5, scoring='accuracy',cv=5,n_jobs=-1)
gsearch5.fit(x_train,y_train)
gsearch5.best_params_, gsearch4.best_score_

In [ ]:
#第七步：降低学习率，增加迭代次数，验证模型
model=lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.01, n_estimators=94, max_depth=3, num_leaves=10,max_bin=175,min_data_in_leaf=101,bagging_fraction=0.6,bagging_freq= 0, feature_fraction= 0.8,
                                                       ambda_l1=0.7,lambda_l2=0.7,min_split_gain=0.1,num_iterations =10000)
model.fit(x_train,y_train)
y_pre=model.predict(x_test)
print("acc:",accuracy_score(y_test,y_pre))



In [ ]:
model.fit(train_df[features],train_df['患有糖尿病标识'])
test_df['label'] = model.predict(test_df[features])
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit.csv', index=None)

## 多折训练与集成

多折cv

In [ ]:
train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')
train_df,test_df = do_nan(train_df,test_df)
train_df,test_df = do_feature(train_df,test_df,False,False)

#划分训练集、验证集
features = [f for f in train_df.columns if f not in ['编号','出生年份','患有糖尿病标识']]
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)

In [ ]:
# 模型交叉验证
def run_model_cv(model, kf, X_tr, y, X_te, cate_col=None):
    train_pred = np.zeros( (len(X_tr), len(np.unique(y))) )
    test_pred = np.zeros( (len(X_te), len(np.unique(y))) )
    cv_clf = []
    for tr_idx, val_idx in kf.split(X_tr, y):
        x_tr = X_tr.iloc[tr_idx]; y_tr = y.iloc[tr_idx]

        x_val = X_tr.iloc[val_idx]; y_val = y.iloc[val_idx]

        call_back = [
            lgb.early_stopping(50),
        ]
        eval_set = [(x_val, y_val)]
        model.fit(x_tr, y_tr, eval_set=eval_set, callbacks=call_back, verbose=-1)

        cv_clf.append(model)

        train_pred[val_idx] = model.predict_proba(x_val)
        test_pred += model.predict_proba(X_te)
        

    test_pred /= kf.n_splits
    return train_pred, test_pred, cv_clf

In [ ]:
#定义模型
clf = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.01, 
                         n_estimators=94, max_depth=3, num_leaves=10,max_bin=175,min_data_in_leaf=101,
                         bagging_fraction=0.6,bagging_freq= 0, feature_fraction= 0.8,lambda_l1=0.7,lambda_l2=0.7,
                         min_split_gain=0.1,num_iterations =5000)

In [ ]:
train_pred, test_pred, cv_clf = run_model_cv(
    clf, KFold(n_splits=4),
    x_train,
    y_train,
    x_test)

test_pred_1 = [ i.argmax() for i in test_pred]
accuracy_score(test_pred_1,y_test)

In [ ]:
train_pred, test_pred, cv_clf = run_model_cv(
    clf, StratifiedKFold(n_splits=4),
    x_train,
    y_train,
    x_test)

test_pred_1 = [i.argmax() for i in test_pred]

accuracy_score(test_pred_1,y_test)

In [ ]:
train_pred, test_pred, cv_clf = run_model_cv(
    clf, StratifiedKFold(n_splits=5),
    train_df[features],
    train_df['患有糖尿病标识'],
    test_df[features])

test_pred_1 = [i.argmax() for i in test_pred]
test_df['label'] = test_pred_1
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit.csv', index=None)

stacking

In [127]:
#使用random forest、extratrees、adaboost、gradientboosting、svm作为第一层分类器
#第二层防止过拟合，用lr

train_df = pd.read_csv('data/比赛训练集.csv', encoding='gbk')
test_df = pd.read_csv('data/比赛测试集.csv', encoding='gbk')
train_df,test_df = do_nan(train_df,test_df)
train_df,test_df = do_feature(train_df,test_df,False,False)

#划分训练集、验证集
features = [f for f in train_df.columns if f not in ['编号','出生年份','患有糖尿病标识']]
x_train, x_test, y_train, y_test = train_test_split(train_df[features],train_df['患有糖尿病标识'],test_size=0.2, random_state=2022)

编号          0.000000
性别          0.000000
出生年份        0.000000
体重指数        0.001578
糖尿病家族史      0.000000
舒张压         0.048718
口服耐糖量测试     0.049704
胰岛素释放实验     0.000000
肱三头肌皮褶厚度    0.000000
患有糖尿病标识     0.000000
dtype: float64
编号          0.000
性别          0.000
出生年份        0.000
体重指数        0.002
糖尿病家族史      0.000
舒张压         0.049
口服耐糖量测试     0.018
胰岛素释放实验     0.000
肱三头肌皮褶厚度    0.000
dtype: float64


<ipython-input-6-da8a2013b959>:44: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  train_df['口服耐糖量测试_diff'] = train_df['口服耐糖量测试'] - train_df.groupby('糖尿病家族史').transform('mean')['口服耐糖量测试']
<ipython-input-6-da8a2013b959>:45: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  test_df['口服耐糖量测试_diff'] = test_df['口服耐糖量测试'] - test_df.groupby('糖尿病家族史').transform('mean')['口服耐糖量测试']


In [128]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [154]:
#底层模型交叉训练oof

def get_oof(model, kf, X_tr, y, X_te, cate_col=None):
    train_pred = np.zeros( (len(X_tr), )) 
    test_pred = np.zeros( (len(X_te), )) 
    cv_clf = []
    for tr_idx, val_idx in kf.split(X_tr, y):
        x_tr = X_tr.iloc[tr_idx]; y_tr = y.iloc[tr_idx]

        x_val = X_tr.iloc[val_idx]; y_val = y.iloc[val_idx]

        model.fit(x_tr, y_tr)

        cv_clf.append(model)

        train_pred[val_idx] = model.predict(x_val)
        test_pred += model.predict(X_te)
        

    test_pred /= kf.n_splits
    return train_pred, test_pred

In [187]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 3,
    'min_samples_leaf': 4,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':150,
    #'max_features': 0.5,
    'max_depth': 6,
    'min_samples_leaf': 4,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 150,
    'learning_rate' : 0.25
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 150,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 4,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [182]:
# Create 5 objects that represent our 5 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=2022, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=2022, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=2022, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=2022, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=2022, params=svc_params)

In [193]:
# Create our OOF train and test predictions. These base results will be used as new features
kf = KFold(n_splits=5)
x_train = train_df[features]
y_train = train_df['患有糖尿病标识']
x_test = test_df[features]
et_oof_train, et_oof_test = get_oof(et,kf, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,kf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada,kf, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,kf,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,kf,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

Training is complete


In [194]:
#第二层的训练及验证数据
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel(),
      'Svc':svc_oof_train.ravel()
    })
base_predictions_test = pd.DataFrame( {'RandomForest': rf_oof_test.ravel(),
     'ExtraTrees': et_oof_test.ravel(),
     'AdaBoost': ada_oof_test.ravel(),
      'GradientBoost': gb_oof_test.ravel(),
      'Svc':svc_oof_test.ravel()
    })

In [197]:
clf = lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metrics='binary_logloss',learning_rate=0.1, 
                         n_estimators=94, max_depth=3, num_leaves=10,max_bin=175,min_data_in_leaf=101,
                         bagging_fraction=0.6,bagging_freq= 0, feature_fraction= 0.8,lambda_l1=0.7,lambda_l2=0.7,
                         min_split_gain=0.1,num_iterations =5000)
clf.fit(base_predictions_train,y_train)
# accuracy_score(clf.predict(base_predictions_test),y_test)
test_df['label'] = clf.predict(base_predictions_test)
test_df.rename({'编号': 'uuid'}, axis=1)[['uuid', 'label']].to_csv('submit.csv', index=None)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] lambda_l1 is set=0.7, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] lambda_l2 is set=0.7, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
